In [1]:
try:
    import openfermion as of
    import openfermionpyscf as ofpyscf
except ImportError:
    print("Installing OpenFermion and OpenFermion-PySCF...")
    !pip install openfermion openfermionpyscf --quiet

import numpy as np
from scipy.sparse import linalg

import cirq
import openfermion as of
import openfermionpyscf as ofpyscf


In [2]:
# Set molecule parameters
geometry = [("H", (0.0, 0.0, 0.0)), ("H", (0.0, 0.0, 0.8))]
basis = "sto-3g"
multiplicity = 1
charge = 0

# Perform electronic structure calculations and
# obtain Hamiltonian as an InteractionOperator
hamiltonian = ofpyscf.generate_molecular_hamiltonian(
    geometry, basis, multiplicity, charge
)

# Convert to a FermionOperator
hamiltonian_ferm_op = of.get_fermion_operator(hamiltonian)

In [3]:
# Map to QubitOperator using the JWT
hamiltonian_jw = of.jordan_wigner(hamiltonian_ferm_op)

# Convert to Scipy sparse matrix
hamiltonian_jw_sparse = of.get_sparse_operator(hamiltonian_jw)

# Compute ground energy
eigs, _ = linalg.eigsh(hamiltonian_jw_sparse, k=1, which="SA")
ground_energy = eigs[0]

print("Ground_energy: {}".format(ground_energy))
print("JWT transformed Hamiltonian:")
print(hamiltonian_jw)

Ground_energy: -1.1341476666770984
JWT transformed Hamiltonian:
(-0.16733398905695257+0j) [] +
(-0.04615669588901532+0j) [X0 X1 Y2 Y3] +
(0.04615669588901532+0j) [X0 Y1 Y2 X3] +
(0.04615669588901532+0j) [Y0 X1 X2 Y3] +
(-0.04615669588901532+0j) [Y0 Y1 X2 X3] +
(0.1625164874887165+0j) [Z0] +
(0.1658325372159041+0j) [Z0 Z1] +
(0.11720364720195851+0j) [Z0 Z2] +
(0.1633603430909738+0j) [Z0 Z3] +
(0.16251648748871647+0j) [Z1] +
(0.1633603430909738+0j) [Z1 Z2] +
(0.11720364720195851+0j) [Z1 Z3] +
(-0.1974429369975581+0j) [Z2] +
(0.1716978839228671+0j) [Z2 Z3] +
(-0.1974429369975581+0j) [Z3]


In [6]:
ops = hamiltonian_jw
dic = ops.terms
tuples = list(ops.terms.keys())
print(tuples)
for typle in tuples:
    for pauli in typle:
        print(pauli[0], pauli[1])
    print('Value: ', dic[typle])

[(), ((0, 'Z'),), ((1, 'Z'),), ((2, 'Z'),), ((3, 'Z'),), ((0, 'Z'), (1, 'Z')), ((0, 'Y'), (1, 'X'), (2, 'X'), (3, 'Y')), ((0, 'Y'), (1, 'Y'), (2, 'X'), (3, 'X')), ((0, 'X'), (1, 'X'), (2, 'Y'), (3, 'Y')), ((0, 'X'), (1, 'Y'), (2, 'Y'), (3, 'X')), ((0, 'Z'), (2, 'Z')), ((0, 'Z'), (3, 'Z')), ((1, 'Z'), (2, 'Z')), ((1, 'Z'), (3, 'Z')), ((2, 'Z'), (3, 'Z'))]
Value:  (-0.16733398905695257+0j)
0 Z
Value:  (0.1625164874887165+0j)
1 Z
Value:  (0.16251648748871647+0j)
2 Z
Value:  (-0.1974429369975581+0j)
3 Z
Value:  (-0.1974429369975581+0j)
0 Z
1 Z
Value:  (0.1658325372159041+0j)
0 Y
1 X
2 X
3 Y
Value:  (0.04615669588901532+0j)
0 Y
1 Y
2 X
3 X
Value:  (-0.04615669588901532+0j)
0 X
1 X
2 Y
3 Y
Value:  (-0.04615669588901532+0j)
0 X
1 Y
2 Y
3 X
Value:  (0.04615669588901532+0j)
0 Z
2 Z
Value:  (0.11720364720195851+0j)
0 Z
3 Z
Value:  (0.1633603430909738+0j)
1 Z
2 Z
Value:  (0.1633603430909738+0j)
1 Z
3 Z
Value:  (0.11720364720195851+0j)
2 Z
3 Z
Value:  (0.1716978839228671+0j)


In [5]:
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
Paulis = [I,X,Y,Z]


    